# Action

> This is functionality for enabling our model to interact with the game world.

In [ ]:
#| default_exp action
#| export
from algorithmic_gamer.types import *
from algorithmic_gamer.fuzzer import *
import ctypes
import time

In [ ]:
#|eval: false
#| export
####TODO test further
def send_input(input_dict):
    # Set the default values for the input values
    x = 0
    y = 0
    trigger_left = 0
    trigger_right = 0
    wButtons = 0

    # Parse the input dictionary
    for key, value in input_dict.items():
        if key == "stick":
            x = int(32767 * value["x"])
            y = int(32767 * value["y"])
        elif key == "trigger":
            if value["side"] == "left":
                trigger_left = int(32767 * value["v"])
            elif value["side"] == "right":
                trigger_right = int(32767 * value["v"])
        elif key == "button":
            if value["state"] == "press":
                if value["name"] == "A":
                    wButtons |= 0x1000
                elif value["name"] == "B":
                    wButtons |= 0x2000
                elif value["name"] == "X":
                    wButtons |= 0x4000
                elif value["name"] == "Y":
                    wButtons |= 0x8000
                elif value["name"] == "LEFT_SHOULDER":
                    wButtons |= 0x0100
                elif value["name"] == "RIGHT_SHOULDER":
                    wButtons |= 0x0200
                elif value["name"] == "BACK":
                    wButtons |= 0x0400
                elif value["name"] == "START":
                    wButtons |= 0x0800
                elif value["name"] == "LEFT_THUMB":
                    wButtons |= 0x0010
                elif value["name"] == "RIGHT_THUMB":
                    wButtons |= 0x0020

    # Create the input structure
    class INPUT(ctypes.Structure):
        _fields_ = [("type", ctypes.c_ulong),
                    ("ki", ctypes.c_ulonglong),
                    ("hi", ctypes.c_ulonglong),
                    ("mi", ctypes.c_ulonglong),
                    ("gp", ctypes.c_ulonglong)]
    input_structure = INPUT(type=2,
                            ki=0,
                            hi=0,
                            mi=0,
                            gp=x | y << 16 | trigger_left << 32 | trigger_right << 48 | wButtons << 16)

    # Send the input
    ctypes.windll.user32.SendInput(1, ctypes.pointer(input_structure), ctypes.sizeof(input_structure))


In [ ]:
#|eval: false
# Test moving the left stick in all four directions
for x in [-1, 0, 1]:
    for y in [-1, 0, 1]:
        print(f"Sending input: x={x}, y={y}")
        send_input({"stick": {"x": x, "y": y}})
        time.sleep(0.5)

# Test pressing both triggers
for trigger_left in [0, 1]:
    for trigger_right in [0, 1]:
        print(f"Sending input: trigger_left={trigger_left}, trigger_right={trigger_right}")
        send_input({"trigger": {"side": "left", "v": trigger_left}, "trigger": {"side": "right", "v": trigger_right}})
        time.sleep(0.5)

# Test pressing all buttons
buttons = ["A", "B", "X", "Y", "LEFT_SHOULDER", "RIGHT_SHOULDER", "BACK", "START", "LEFT_THUMB", "RIGHT_THUMB",]

for button in buttons:
    print(f"Sending input: {button} button pressed")
    send_input({"button": {"name": button, "state": "press"}})
    time.sleep(0.5)

print("Test complete")


Sending input: x=-1, y=-1
Sending input: x=-1, y=0
Sending input: x=-1, y=1
Sending input: x=0, y=-1
Sending input: x=0, y=0
Sending input: x=0, y=1
Sending input: x=1, y=-1
Sending input: x=1, y=0
Sending input: x=1, y=1
Sending input: trigger_left=0, trigger_right=0
Sending input: trigger_left=0, trigger_right=1
Sending input: trigger_left=1, trigger_right=0
Sending input: trigger_left=1, trigger_right=1
Sending input: A button pressed
Sending input: B button pressed
Sending input: X button pressed
Sending input: Y button pressed
Sending input: LEFT_SHOULDER button pressed
Sending input: RIGHT_SHOULDER button pressed
Sending input: BACK button pressed
Sending input: START button pressed
Sending input: LEFT_THUMB button pressed
Sending input: RIGHT_THUMB button pressed
Test complete


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()